## visualize image dataset (don't run this)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have defined the build method as shown in the provided code

# Set the image_set and args accordingly
image_set = "train"
coco_path = "./coco_dataset_test"

# Create the dataset using the build method
dataset = build(image_set, coco_path, strong_aug=True, fix_size=False)

# Iterate over the dataset to print images and annotations
for i in range(len(dataset)):
    # Get the image and target (annotations)
    image, target = dataset[i]
    
    # Convert image tensor to numpy array
    image_np = image.permute(1, 2, 0).numpy()

    # Plot the image
    plt.imshow(image_np)
    plt.axis('off')
    plt.title(f'Image {i}')
    plt.show()

    # Print annotations
    print("Annotations:")
    print(target)
    break


## show image (don't run this)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have defined the build method as shown in the provided code

# Set the image_set and args accordingly
image_set = "train"
coco_path = "./coco_dataset_test"

# Create the dataset using the build method
dataset = build(image_set, coco_path, strong_aug=True, fix_size=False)
dataloader_train = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0, collate_fn=utils.collate_fn)

for batch in dataloader_train:
    images, targets = batch
    #targets =[ target.to(device) for target in targets]
    #images = list(img.to(device) for img in images)
    print("targets",targets)
     

    print(F"targets=", targets)
    print(F"images=", images)
        # Convert image tensor to numpy array
    # Iterate over the images in the NestedTensor
    images_list = images.tensors
    for i, image in enumerate(images_list):
        # Get the image
        
        image_np = image.permute(1, 2, 0).numpy()

        # Plot the image
        plt.imshow(image_np)
        plt.axis('off')
        plt.title(f'Image {i}')
        plt.show()
        
    #for i, image in enumerate(images):
        #print(f"Shape of image {i}: {image.shape}")
        
    #images_tensor = torch.stack(images)
    #print(images_tensor.shape)
    
     # Get color images (assuming images are in RGB format)
    #color_images = [F.to_pil_image(image.cpu()) for image in images]

    # Display color images
    #for i, color_image in enumerate(color_images):
        #plt.imshow(color_image)
        #plt.show()

    # Access tensors (boxes, labels, image_ids, ids) for further processing
    #for target in targets:
        #Access target data, e.g., boxes, labels, etc.
        #boxes = target[:, :4]
        #labels = target[:, 4]
        #image_ids = target[:, 5]
        #ids = target[:, 6]
        #print(boxes, labels, image_ids, ids)
    break

# Iterate over the dataset to print images and annotations
for i in range(len(dataset)):
    # Get the image and target (annotations)
    image, target = dataset[i]
    
    # Convert image tensor to numpy array
    image_np = image.permute(1, 2, 0).numpy()

    # Plot the image
    plt.imshow(image_np)
    plt.axis('off')
    plt.title(f'Image {i}')
    plt.show()

    # Print annotations
    print("Annotations:")
    print(target)
    break


## Visualize Pretrain resnet50 Model (don't run this)

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define your arguments
class Args:
    def __init__(self):
        self.backbone = "resnet50"  # or any other supported backbone
        self.lr_backbone = 0.1  # example learning rate for backbone, adjust as needed
        self.masks = False  # whether to include masks in the output
        self.num_feature_levels = 1  # number of feature levels, adjust as needed
        self.dilation = False  # whether to use dilation in backbone
        self.hidden_dim = 256  # hidden dimension for transformer
        self.position_embedding = "sine"  # type of position embedding to use

args = Args()

# Build the backbone model
model =build_backbone(args).to(device)

# Assuming you have input data 'input_data', pass it through the model
# input_data should be a NestedTensor, which is a tensor with an associated mask
# Example usage assuming input_data is properly defined
#output = model(nested_images)

with torch.no_grad():
    all_extracted_features = []
    all_outputs = []

    for batch in dataloader_train:
        images, targets = batch
        targets = [target.to(device) for target in targets]
        images = [img.to(device) for img in images]
        #print("shape of images",images[0].shape)
        #for img in images:
            #print(img.shape)
            #break
        mask = [torch.ones_like(img) for img in images]
        nested_images = nested_tensor_from_tensor_list(images).to(device)
        output,extracted_features = model(nested_images)
        #print("shape of extracted_features")
        #for img in extracted_features:
            #print(img.shape)
            #break
        
        #print("shape of output")
        #for img in output:
            #for i in img.tensors:
                #print(i.shape)
                #break
            
        #all_extracted_features.append(extracted_features.detach())
         # Append the output and extracted features to the lists
        all_outputs.append(output)
        all_extracted_features.append(extracted_features)
            # Clear the memory
        del images, targets, output, extracted_features
        torch.cuda.empty_cache()

    # Save the lists of outputs and extracted features
    torch.save(all_outputs, os.path.join(os.getcwd(), 'all_outputs.pt'))
    torch.save(all_extracted_features, os.path.join(os.getcwd(), 'all_extracted_features.pt'))
        
    #print("shape of image",images)
    # Concatenate features from all batches
    #all_extracted_features = torch.cat(all_extracted_features, dim=0)

#print("Shape of all extracted features:", all_extracted_features.shape)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the lists of outputs and extracted features in cpu memory
all_outputs = torch.load(os.path.join(os.getcwd(), 'all_outputs.pt'), map_location='cpu')
all_extracted_features = torch.load(os.path.join(os.getcwd(), 'all_extracted_features.pt'), map_location='cpu')

# Now you can use all_outputs and all_extracted_features in your code

In [ ]:
for i, batch in enumerate(all_extracted_features):
    for j, tensor in enumerate(batch):
        print(f"Shape of tensor {j} in batch {i}: {tensor.shape}")
        

In [ ]:
print("shape of all_outputs",all_extracted_features)

In [ ]:
print("shape of all_outputs",all_outputs)

## Reshape All Extracted Feature (don't run this)

In [ ]:
reshaped_features = []

for batch in all_extracted_features:
    reshaped_batch = [tensor.permute(2, 3, 0, 1).contiguous().view(-1, *tensor.shape[:2]) for tensor in batch]
    reshaped_features.append(reshaped_batch)

In [ ]:
for i, batch in enumerate(reshaped_features):
    for j, tensor in enumerate(batch):
        print(f"Shape of tensor {j} in batch {i}: {tensor.shape}")

## Visualize encoder layer output (don't run this)

In [ ]:
#device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
encoder_layer = TransformerEncoderLayer(d_model=256, nhead=8, dim_feedforward=2048, dropout=0.1, activation="relu", normalize_before=False)

# Assume reshaped_features is your list of reshaped image features
for i, batch in enumerate(reshaped_features):
    for j, src in enumerate(batch):
        
        # Pass the tensor through the encoder layer
        output = encoder_layer(src)

        # Print the output
        print(f"Output of tensor {j} in batch {i}: {output}")
                # Print the shape of the output
        print(f"Shape of output: {output.shape}")       
        

for i in range(output.shape[1]):
    plt.figure(figsize=(10,10))
    plt.imshow(output.detach().numpy()[i], cmap='hot', interpolation='nearest')
    plt.show()